# Environment Setup

We will require a Conda environment in order to import the `ViennaRNA` library created by [Lorenz et al. (2011)](https://doi.org/10.1186/1748-7188-6-26).

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


With the Conda environment now installed, we can install the `ViennaRNA` library.

**NOTE**: Colab will crash after this install. Run the subsequent cells manually by clicking an appropriate option under the **Runtime** menu.

In [ ]:
!conda install -c bioconda viennarna

# Imports

In [ ]:
import RNA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

# Functions

In [ ]:
def find_hybridization_energy(sequence1, sequence2, temp):

    RNA.cvar.temperature = temp

    # First, we concatenate the two RNA sequences using the '&' symbol
    hybrid_sequence = RNA.fold_compound(f"{sequence1}&{sequence2}")

    # Finally, we use the ViennaRNA library's inbuilt mfe_dimer function to compute the hybridization energy
    structure, energy = hybrid_sequence.mfe_dimer()

    return energy

# New section

In [ ]:
def find_spacing(rRNA,rbs,temp,gram):

  RNA.cvar.temperature = temp
  min_energy = 0
  sd = ""
  best_spacing = 0

  for i in range(len(rbs)-len(rRNA)+1):

    energy = find_hybridization_energy(rRNA,rbs[i:i+len(rRNA)],temp)
    spacing = len(rbs[i+len(rRNA):])
    penalty = 1

    if gram == "Positive":
      opt_spacing = 9
      if spacing < opt_spacing:
        penalty = np.exp(-0.5*(((spacing - opt_spacing)**2)/1))                   # Punishing lower spacing more for Gram-positives
      elif spacing > opt_spacing:
        penalty = np.exp(-0.5*(((spacing - opt_spacing)**2)/2))
    elif gram == "Negative":
      opt_spacing = 7
      if spacing < opt_spacing:
        penalty = np.exp(-0.5*(((spacing - opt_spacing)**2)/2))
      elif spacing > opt_spacing:
        penalty = np.exp(-0.5*(((spacing - opt_spacing)**2)/1))                   # Punishing higher spacing more for Gram-negatives

    energy = energy * penalty

    if energy <= min_energy:
      min_energy = energy
      sd = rbs[i:i+len(rRNA)]
      best_spacing = spacing

  return find_hybridization_energy(rRNA,sd,temp), sd, best_spacing

In [ ]:
def find_au_score(rbs,sd):

    sd_loc = rbs.index(sd)
    upstream = rbs[sd_loc-11 : sd_loc]

    au_score = upstream.count("A") + upstream.count("U")

    if len(upstream) == 0:
      return 0
    else:
      return au_score / len(upstream)

In [ ]:
def find_accessibility_score(rbs,cds,sd,temp):

  RNA.cvar.temperature = temp

  sd_loc = rbs.index(sd)

  upstream = rbs[-27:]
  downstream = cds[:54-len(upstream)]

  structure, fold_energy = RNA.fold(upstream + downstream)

  loop_count = structure.count(".")                                               # Number of unpaired nucleotides
  stack_count = structure.count("(") + structure.count(")")                       # Number of paired nucleotides

  accessibility_score = loop_count / (loop_count + stack_count)

  return structure, accessibility_score, fold_energy

In [ ]:
def find_standby_score(rbs,structure):

  upstream = rbs[-27:]
  upstream_structure = structure[-27:]
  rRNA_length = 8

  if len(upstream_structure) != 0:

    upstream = upstream[::-1]
    upstream_structure = upstream_structure[::-1]

    best_gap = 0
    best_accessibility = 0

    for i in range(len(upstream)-rRNA_length+1):

      gap = i+rRNA_length
      loop_count = upstream_structure[i:i+rRNA_length].count(".")
      stack_count = upstream_structure[i:i+rRNA_length].count("(") + upstream_structure[i:i+rRNA_length].count(")")
      accessibility = loop_count / (loop_count + stack_count)

      if accessibility > best_accessibility:
        best_gap = gap
        best_accessibility = accessibility

    return best_gap, best_accessibility

  else:

    return 0,0

In [ ]:
codon_scores = {"AGC":0,"CCG":0.78,"CAG":0.95,"UGC":1.01,"CUC":1.1,"UCC":1.1,"CCC":1.17,"UAG":1.19,"CGG":1.2,"CUU":1.21,"CCU":1.26,"UUC":1.29,"CCA":1.3,"UAC":1.3,"CGC":1.31,"UCU":1.32,"ACC":1.32,"ACA":1.33,"CGU":1.33,"GCC":1.33,"CUA":1.34,"AAC":1.37,"CAC":1.41,"UCA":1.41,"AAA":1.43,"GCU":1.45,"AGU":1.46,"UAA":1.47,"GGC":1.48,"AGA":1.51,"UUU":1.54,"GAC":1.54,"AAG":1.56,"ACU":1.56,"UGA":1.56,"UGU":1.59,"CAA":1.59,"GUC":1.61,"GCA":1.64,"UGG":1.66,"CGA":1.69,"AGG":1.71,"GGA":1.71,"GGU":1.71,"GAG":1.72,"UUA":1.72,"UCG":1.78,"GGG":1.79,"GCG":1.84,"ACG":1.88,"GAU":1.92,"AAU":1.93,"GAA":2.02,"GUU":2.1,"GUA":2.11,"UAU":2.16,"CAU":2.17,"AUC":2.41,"AUU":2.57,"CUG":2.63,"AUA":2.73,"UUG":4.18,"GUG":4.22,"AUG":4.3}

In [ ]:
def find_codon_score(cds):

  codon = cds[:3]
  codon_score = codon_scores[codon]

  return codon_score

# Model

We load the trained machine learning model using `joblib`.

**NOTE**: Any files will have to be loaded manually every time. Google Colab does not store the files permanently.

In [ ]:
model = joblib.load("iGEM 2023 - Final Model.joblib")

In [ ]:
def find_tir(gram_stain,temperature,rRNA,RBS,CDS):

  rRNA = rRNA.upper().replace("T","U")
  RBS = RBS.upper().replace("T","U")
  CDS = CDS.upper().replace("T","U")
  rRNA = rRNA[-8:]

  binding_energy, shine_dalgarno, spacing = find_spacing(rRNA,RBS,temperature,gram_stain)
  au_score = find_au_score(RBS,shine_dalgarno)
  structure, accessibility_score, folding_energy = find_accessibility_score(RBS,CDS,shine_dalgarno,temperature)
  standby_gap, standby_accessibility = find_standby_score(RBS,structure)
  codon_score = find_codon_score(CDS)

  features = np.array([binding_energy, spacing, au_score, accessibility_score, folding_energy, standby_gap, standby_accessibility, codon_score]).reshape(1,-1)
  df = pd.DataFrame(features)
  df.columns = ["Binding energy","Spacing","AU score","Accessibility score","Folding energy","Standby gap","Standby accessibility","Codon score"]

  tir = model.predict(df)[0]

  return tir

In [ ]:
# Sample input

gram = "Negative"
temp = 37
rRNA = "CCUCCUUA"
rbs = "UUCUAGAGUGCAUAAGGAGUGCUCG"
cds = "AUGUCCAGAUUAGAUAAAAGUAAAGUGAUGGCGAGCUCUGAAGACGUUAUCAAAGAGUUCAUGCGUUUCAAAGUUCGUAUGGAAGGUUCCGUUAACGGUCACGAGUUCGAAAUCGAAGGUGAAGGUGAAGGUCGUCCGUACGAAGGUACCCAGACCGCUAAACUGAAAGUUACCAAAGGUGGUCCGCUGCCGUUCGCUUGGGACAUCCUGUCCCCGCAGUUCCAGUACGGUUCCAAAGCUUACGUUAAACACCCGGCUGACAU"

find_tir(gram, temp, rRNA, rbs, cds)

4.227431973161189

# Optimization Algorithm

Random Mutator function :
issues with code : currently replaces all nucleotides of one type instead of individual ones at certain locations, also doesnt take num_mutate(no. of nucleotides to be mutated as a parameter)

In [ ]:

import random

def random_mutator(RBS, untouched_bases, number_of_bases_to_change):
    sequence_length = len(RBS)
    mutation_indices = random.sample(list(range(sequence_length)), number_of_bases_to_change)
    mutated_sequence = list(RBS)

    for index in mutation_indices:
        if index not in untouched_bases :
          current_base = mutated_sequence[index]
          new_base = random.choice(['A', 'U', 'G', 'C'])  # Randomly select a new base
          while new_base == current_base:
              new_base = random.choice(list({'A', 'U', 'G', 'C'}-{current_base}))
          mutated_sequence[index] = new_base

    new_RBS=''.join(mutated_sequence)
    return new_RBS
#print(random_mutator('ACUG',4))

Cost function :

In [ ]:
def cost(desired_tir,current_RBS):

  cost = abs((desired_tir)-(find_tir(gram_stain,temperature,rRNA,current_RBS,CDS)))
  return cost


weights(T) function

In [ ]:
import math
def weights(desired_tir,solution,T): #higher order function taking cost function as a parameter
  choice_list=[solution,new_RBS]
  delta_cost=abs(cost(desired_tir,solution)-cost(desired_tir,new_RBS))
  if cost(desired_tir,new_RBS)<cost(desired_tir,solution) :
    solution=new_RBS
  else :
    solution = random.choices(choice_list,[1,math.exp(-((delta_cost))/T)])[0]
  return solution

loop :

In [ ]:
import random
import math
#T=6
# gram_stain = input('is the organism gram +ve or gram -ve\n')
# temperature = input('what is the temperature\n')
# rRNA = input('input rRNA binding sequence\n')
# CDS = input('input current cds sequnce\n')
# RBS = input('input current RBS sequence\n')
gram_stain = "Positive"
temperature = 30
rRNA = "CTAAGGAA"
RBS = "GTTCGAAGGAACTACAAAATAAATTATAAGGAGGCACTCA"
untouched_bases =  [0,27,28,29,30,31,32,33]
CDS = "ATGGTTTCTAAAGGTGAAGAATTATTTACAGGAGTTGTTCCTATTTTAGTTGAACTTGATGGTGATGTTAATGGACATAAATTTTCAGTTTCTGGAGAAGGTGAAGGAGATGCTACTTATGGAAAACTTACTCTTAAATTTATTTGTACAACTGGAAAACTTCCAGTTCCTTGGCCAACATTAGTTACAACTCTTACTTATGGTGTTCAATGTTTTTCTCGTTATCCTGATCACATGAAACAACATGATTTCTTTAAATCAGCTATGCCAGAAGGTTATGTTCAAGAACGTACTATTTTCTTTAAAGATGATGGAAATTATAAAACACGTGCAGAAGTTAAATTTGAAGGTGATACTTTAGTTAATCGTATTGAACTTAAAGGAATTGATTTTAAAGAAGATGGTAATATTTTAGGACATAAACTTGAATATAATTATAATTCTCATAATGTTTATATTATGGCTGATAAACAAAAGAATGGTATTAAAGTTAATTTTAAAATTCGTCATAATATTGAAGATGGATCAGTTCAATTAGCAGATCATTATCAACAAAATACACCTATTGGTGATGGACCTGTTCTTCTTCCAGATAATCATTATTTATCAACTCAATCTAAACTTTCAAAAGATCCAAATGAAAAACGTGATCACATGGTTTTACTTGAATTTGTTACAGCTGCTGGAATTACTTTAGGAATGGATGAACTTTATAAATAA"
 #input('input desired rate of translation initiation\n')
number_of_bases_to_change= 14
new_RBS=('')
solution = RBS
best_solution= RBS     #lol
best_best_solution=RBS #lmaoo
for x in [1000.0]:
  RBS = "GTTCGAAGGAACTACAAAATAAATTATAAGGAGGCACTCA"
  desired_tir= x
  for i in range(10):#thanks tp karthik for helping me debug this piece of code ilysm
      T=100000
      while T>0 :

        RBS = best_solution
        new_RBS = random_mutator(RBS, untouched_bases,11)

        while 'AUG' in str(new_RBS[2:33]+new_RBS[41:]):
           new_RBS = random_mutator(RBS, untouched_bases,11)
        RBS = weights(desired_tir,solution,T)
        RBS = RBS.replace('T','U')

        if cost(desired_tir,RBS)<cost(desired_tir,best_solution):
          best_solution=RBS
        #print(find_tir(gram_stain,temperature,rRNA,RBS,CDS))
        T=T-1000
      #print(f'the solution is {best_solution} tir = {find_tir(gram_stain,temperature,rRNA,best_solution,CDS)}')

      if cost(desired_tir,best_solution)<cost(desired_tir,best_best_solution):
          best_best_solution=best_solution
  print(f'for {desired_tir} the best solution is {best_best_solution} tir = {find_tir(gram_stain,temperature,rRNA,best_best_solution,CDS)}')
  print(best_best_solution.find('AUG'))

for 1000.0 the best solution is GAGUCCCGUCAAUUCAACACGGUUCCUAAGGAGGGUCAAC tir = 4.564656424374677
-1


In [ ]:
import random

In [ ]:
for i in range(200000):
  DNA = ""
  for i in range(40):
    DNA += random.choice(["A","T","G","C"])
  print(DNA + ' - ' + str(random.uniform(0,100)))

Streaming output truncated to the last 5000 lines.
GAAAACGCTTGCTTGCAGCGTCCATTTACGACCTTGTCGG - 71.73119308471978
TACGTGTCTGAGGCGACGCTGCGACTGGATACAGCTCGAG - 20.783953802448252
TTATGGGTGCAGGTTCTGGATGAGAATATTATCAGGCTGC - 39.16828467867084
AGACACATCTTCAGTTCCAGTACTTTCTACGTGACTAGTT - 43.2084947707374
AAGGGTCCTAGGGGGGTTTCGACAACTTCCGCCCCAAGGG - 55.94234964325307
TTCAGATCCATAGTAGTGTTTAGTCGAGTCCCCCTTGATC - 2.368467174484057
TCAACCCCGAATGGGCACGACCAAGATAGGAGACTAAGCT - 77.08209930054971
TGTTTAGCCGAACATCCTTCCAGCCTAAGGCTGGTCGGTG - 61.23136981768459
AATTTGTCACCTTGCTACCCAGGGGGAGGCACATCCCGTA - 58.119603843769916
TGCGTGACAGCATGGGCTTGTTATTTTTTGTTCAGTCTGG - 44.244200678985045
CGGCTGCACGTCTAGTGATGACCGCTACCGAAGCAATTCA - 74.12164926548083
TAGTGGTTCTTGACTAGCCTTAACCCATGATAGGGTATTG - 63.56430134736486
CAAGAATCAATAGTCACGGCATAAGGGCTGAATACCTCCT - 70.68143764633118
TATGCATGGTCTACTCAGGTCCGACAGGAAGGTAACGGAG - 71.20801086729006
AATTAGCGCAGGCGTCTAGCCCTTTGGAACACGGCGACGG - 73.1633530374278
TTACGGCTAATCAAGTTATGCCGACGTCGACTG

KeyboardInterrupt: ignored

In [ ]:
\